In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import cluster, preprocessing, linear_model, tree, model_selection, feature_selection
from sklearn import base, ensemble, decomposition, metrics, pipeline, datasets, impute

from functools import partial
import os
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import catboost as cb
from sklearn import ensemble, preprocessing, tree, model_selection, feature_selection, pipeline, metrics, svm
from sklearn.model_selection import StratifiedKFold
from sklearn import impute, pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [147]:
train=pd.read_csv('Train_aBjfeNk.csv')
test=pd.read_csv('Test_LqhgPWU.csv')

In [148]:
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
rev_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

In [149]:
train_copy  = train.copy()
test_copy = test.copy()
train_copy['tr'] = 1
test_copy['tr'] = 0

appended = pd.concat([train_copy, test_copy], axis = 0)

cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
label_enc = {}
for col in cat_cols:
    appended[col] = appended[col].astype(str)
    enc = preprocessing.LabelEncoder().fit(appended[col])
    appended[col] = enc.transform(appended[col])
    label_enc[col] = enc

cats = ['Gender', 'Ever_Married','Graduated','Profession','Spending_Score',
'Var_1']
appended = pd.get_dummies(appended, columns = cats)
##################### create features from ID column ##############
def id_features(data):
    df = data.copy()
    df['week'] = df['ID']%7
    df['month'] = df['ID']%30
    df['year'] = df['ID']%365
    df['num_weeks'] = df['ID']//7
    df['num_year'] = df['ID']//365
    df['num_quarter'] = df['ID']//90
    df['quarter'] = df['ID']%90
    df['num_days'] = df['ID'].values - 458982
    df['num_weeks_2'] = (df['ID'].values - 458982)//7
    df['num_months_2'] = (df['ID'].values - 458982)//30

    return df

def id_features2(data):
    df = data.copy()
    df['week'] = df['ID']%7
    df['month'] = df['ID']%30
    df['year'] = df['ID']%365
    df['quarter'] = df['ID']%90


    return df
appended = id_features(appended)
#appended = pd.get_dummies(appended, columns = cat_cols)
train_copy = appended.loc[appended['tr'] == 1]
test_copy = appended.loc[appended['tr'] == 0]
Xcols = appended.drop(['Segmentation', 'tr'], axis = 1).columns
'''Xcols = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1']'''
ycol = 'Segmentation'

X = train_copy[Xcols]
y = train_copy[ycol]

Xtest = test_copy[Xcols]

In [150]:
X

,ID,Age,Work_Experience,Family_Size,Gender_0,Gender_1,Ever_Married_0,Ever_Married_1,Ever_Married_2,Graduated_0,...,week,month,year,num_weeks,num_year,num_quarter,quarter,num_days,num_weeks_2,num_months_2
0,462809,22,1.0,4.0,0,1,1,0,0,1,...,4,29,354,66115,1267,5142,29,3827,546,127
1,462643,38,NaN,3.0,1,0,0,1,0,0,...,6,13,188,66091,1267,5140,43,3661,523,122
2,466315,67,1.0,1.0,1,0,0,1,0,0,...,3,25,210,66616,1277,5181,25,7333,1047,244
3,461735,67,0.0,2.0,0,1,0,1,0,0,...,1,5,10,65962,1265,5130,35,2753,393,91
4,462669,40,NaN,6.0,1,0,0,1,0,0,...,4,9,214,66095,1267,5140,69,3687,526,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,22,0.0,7.0,0,1,1,0,0,1,...,2,8,103,66288,1271,5155,68,5036,719,167
8064,464685,35,3.0,4.0,0,1,1,0,0,1,...,4,15,40,66383,1273,5163,15,5703,814,190
8065,465406,33,1.0,1.0,1,0,1,0,0,0,...,4,16,31,66486,1275,5171,16,6424,917,214
8066,467299,27,1.0,4.0,1,0,1,0,0,0,...,0,19,99,66757,1280,5192,19,8317,1188,277


In [151]:
X.isna().sum()

ID                    0
Age                   0
Work_Experience     829
Family_Size         335
Gender_0              0
Gender_1              0
Ever_Married_0        0
Ever_Married_1        0
Ever_Married_2        0
Graduated_0           0
Graduated_1           0
Graduated_2           0
Profession_0          0
Profession_1          0
Profession_2          0
Profession_3          0
Profession_4          0
Profession_5          0
Profession_6          0
Profession_7          0
Profession_8          0
Profession_9          0
Spending_Score_0      0
Spending_Score_1      0
Spending_Score_2      0
Var_1_0               0
Var_1_1               0
Var_1_2               0
Var_1_3               0
Var_1_4               0
Var_1_5               0
Var_1_6               0
Var_1_7               0
week                  0
month                 0
year                  0
num_weeks             0
num_year              0
num_quarter           0
quarter               0
num_days              0
num_weeks_2     

In [152]:
feat=['Age','Profession_0','ID','Profession_0','year','quarter','month']
scaler=StandardScaler()
km=KMeans(n_clusters=15)
km.fit(scaler.fit_transform(X[feat]))
X['cluster']=km.predict(scaler.fit_transform(X[feat]))
Xtest['cluster']=km.predict(scaler.fit_transform(Xtest[feat]))

/home/omar/Desktop/chatboot project/omar/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/omar/Desktop/chatboot project/omar/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [153]:
appended = pd.concat([X,Xtest], axis = 0)
cat_cols = ['cluster']
label_enc = {}
for col in cat_cols:
    appended[col] = appended[col].astype(str)
    enc = preprocessing.LabelEncoder().fit(appended[col])
    appended[col] = enc.transform(appended[col])
    label_enc[col] = enc

cats = ['cluster']
appended = pd.get_dummies(appended, columns = cats)

In [154]:
X=appended.iloc[:X.shape[0]]
Xtest=appended.iloc[X.shape[0]:]

# Models:

In [155]:
model1 = pipeline.make_pipeline(impute.KNNImputer(n_neighbors = 10), ensemble.RandomForestClassifier(class_weight = 'balanced_subsample',
                    n_estimators = 200, max_depth =16, criterion = 'entropy', max_features = .8, oob_score = True, random_state = 0))

In [156]:
modelrf2 = pipeline.make_pipeline(impute.KNNImputer(n_neighbors = 8), ensemble.RandomForestClassifier(class_weight = 'balanced_subsample',
                    n_estimators = 250, max_depth =24, criterion = 'entropy', max_features = .8, oob_score = True, random_state = 0))

In [157]:
model2 = pipeline.make_pipeline(impute.SimpleImputer(), ensemble.RandomForestClassifier(class_weight = 'balanced_subsample',
                    n_estimators = 200, max_depth =20, criterion = 'entropy', max_features = .8, oob_score = True, random_state = 0))

In [158]:
model3 = lgb.LGBMClassifier(n_estimators=300, max_features = .8, max_depth = 20, learning_rate = 1.1)

In [159]:
model4=xgb.XGBClassifier(n_estimators=500,max_depth=22,eta=0.1,colsample_bytree=0.8)


In [193]:
model5=xgb.XGBClassifier(n_estimators=500,max_depth=22,eta=0.1,colsample_bytree=0.75,subsample=0.9)

In [194]:
model6=xgb.XGBClassifier(n_estimators=600,max_depth=22,eta=0.1,colsample_bytree=0.8,subsample=0.8)

In [195]:
model7=pipeline.make_pipeline(impute.KNNImputer(n_neighbors = 8),model5)


In [163]:
mod=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=22,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=600, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [164]:
mod1=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.75, eta=0.1, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=22,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=1200, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [174]:
mod3=lgb.LGBMClassifier(n_estimators=500, max_features = .85, max_depth = 15, learning_rate = 1.1)

# Submission:

In [175]:
mod3.fit(X,y)

LGBMClassifier(learning_rate=1.1, max_depth=15, max_features=0.85,
               n_estimators=500)

In [176]:
model=mod3
pred = pd.DataFrame()
pred['ID'] = test['ID'].values
pred['Segmentation'] = pd.Series(model.predict(Xtest))
pred.to_csv('Seg_14.csv', index = None)

In [181]:
from skopt import gp_minimize, space, gbrt_minimize, dummy_minimize, forest_minimize

In [182]:

def optimize_sk(params, param_names, X, y, scoring, estimator, cv = model_selection.StratifiedKFold(n_splits = 5)):
    '''params: list of param values
    param_names: param names
    x: training exogs
    y: training endogs
    return: negative metric after k fold validation'''

    params = dict(zip(param_names, params))

    # Initialize the model
    model = estimator(**params)

    kf = cv

    scores = []
    for train_index, test_index in kf.split(X, y):
        # Split Data
        X_train, y_train = np.array(X)[train_index, :], y[train_index]
        X_test, y_test = np.array(X)[test_index, :], y[test_index]

        # Fit model
        im = impute.KNNImputer().fit(X_train)
        X_train = im.transform(X_train)
        model.fit(X_train, y_train)

        # Evaluate model
        preds = model.predict(im.transform(X_test))
        scores.append(scoring(y_test, preds))

    return -np.mean(scores)

# Scoring
def f1_score(y_true, y_pred):
    return metrics.f1_score(y_true, y_pred, average = 'macro')

def accuracy(y_true, y_pred):
    return metrics.accuracy_score(y_true, y_pred)

# Parameter Space
param_space = [
    space.Integer(100, 1000, name = 'n_estimators'),
    space.Integer(2, 25, name = 'max_depth'),
    space.Real(0, 1, name = 'max_features'),
    space.Integer(2, 25, name = 'min_samples_leaf'),
    space.Categorical(['gini', 'entropy'], name = 'criterion'),
    space.Categorical([None, 'balanced', 'balanced_subsample'], name = 'class_weight'),
    space.Categorical([True, False], name = 'bootstrap')
]

# Param names
names = ['n_estimators', 'max_depth', 'max_features', 'min_samples_leaf', 'criterion', 'class_weight', 'bootstrap']

cat_cols =  ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'ID']
cat_cols =  ['Var_1']

# Define objective - reformat it in terms of what is required for skopt
objective_optimization = partial(optimize_sk, param_names = names, X = X, y = y, 
                                scoring = accuracy, estimator = partial(ensemble.RandomForestClassifier, n_jobs = -1, random_state = 0))

# Perform Optimization
#gbrt_minimize, dummy_minimize, forest_minimize
'''skopt_optimization = gp_minimize(func = objective_optimization, 
                                dimensions = param_space, n_calls = 10, n_random_starts = 10, 
                                x0 = None, y0 = None, random_state = 10, 
                                verbose = 10)'''
skopt_optimization = dummy_minimize(func = objective_optimization, 
                                dimensions = param_space, n_calls = 10,
                                x0 = None, y0 = None, random_state = 10, 
                                verbose = 10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 35.0973
Function value obtained: -0.5255
Current minimum: -0.5255
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 93.7886
Function value obtained: -0.5332
Current minimum: -0.5332
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 56.9114
Function value obtained: -0.5269
Current minimum: -0.5332
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 163.2877
Function value obtained: -0.5297
Current minimum: -0.5332
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 38.7924
Function value obtained: -0.5362
Current minimum: -0.5362
Iteration No: 6 sta

In [183]:
model = pipeline.make_pipeline(impute.KNNImputer(), 
                               ensemble.RandomForestClassifier(**dict(zip(names, skopt_optimization.x)), 
                                                               n_jobs = -1, random_state = 0)).fit(X, y)

In [184]:
model=model
pred = pd.DataFrame()
pred['ID'] = test['ID'].values
pred['Segmentation'] = pd.Series(model.predict(Xtest))
pred.to_csv('Seg_15.csv', index = None)

In [185]:
dict(zip(names, skopt_optimization.x))

{'n_estimators': 442,
 'max_depth': 14,
 'max_features': 0.1054018334878011,
 'min_samples_leaf': 6,
 'criterion': 'gini',
 'class_weight': None,
 'bootstrap': False}

In [186]:
model44 = pipeline.make_pipeline(impute.KNNImputer(n_neighbors = 10), ensemble.RandomForestClassifier(class_weight = 'balanced_subsample',
                    n_estimators = 422, max_depth = 14, criterion = 'entropy', max_features = .105, oob_score = True, random_state = 0)).fit(X, y)

In [187]:
model=model44
pred = pd.DataFrame()
pred['ID'] = test['ID'].values
pred['Segmentation'] = pd.Series(model.predict(Xtest))
pred.to_csv('Seg_16.csv', index = None)